# Support Vector Machine


Voorbeeld oplossing van een klassificatie probleem met ML-model **Support Vector Machine**.

Dataset: Titanic 
- `test.csv`: de testdata set
- `train.csv`: de traindata set


In [1]:
#All imports:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
# helpers
import os
import pandas as pd

def load_csv_pd_data(path, filename):
    '''
    load_csv_pd_data() - construct a pandas DataFrame object 
           from data in a Excel csv-file `filename`,
           stored in a folder `path`.
    @returns: a pandas DataFrame
    '''
    csv_path = os.path.join(path, filename)
    # DEBUG: print(csv_path)
    return pd.read_csv(csv_path)

### De data
Er zijn 2 data sets klaar gezet, één om het model te trainen en één om mee te testen. De set om mee te trainen heeft al de overlevings voorspelling per passagier terwijl de test set deze niet heeft 

In [3]:
train_df = load_csv_pd_data('./data/', 'train.csv')
test_df = load_csv_pd_data('./data/', 'test.csv')
combine= [train_df, test_df]

In [4]:
print(train_df.columns.values)
#print('-'*40)
#print(test_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


### Feature selection

In Jupyter notebook `Feature_Selection_analysis_Titanic` is een gedegen analyse gedaan om *features* te selecteren die meegenomen worden in het classificatie probleem wel/niet overleven van de Titanic. De resultaten staan in de volgende cell. Zie voor details de genoemde Jupyter notebook.

In [5]:
# ———————————————————————
# verwijderen `Cabin` en `Ticket`
train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

# ———————————————————————
#  Aanmaken nieuwe features title
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

# Een aantal van deze titels samenvoegen:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

# Omzetten van de categorische titel waarden in ordinale waarden:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

# train_df.head()

# ———————————————————————
# Nu het verwijderen van features Name en PassengerId
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]

# train_df.shape, test_df.shape

# ———————————————————————
# Sex male/female omzetten in 0/1.
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

# train_df.head()

# ———————————————————————
# Aanvullen van ontbrekende waarden Age, Sex and Pclass
guess_ages = np.zeros((2,3))

for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]
                    
    dataset['Age'] = dataset['Age'].astype(int)

# train_df.head()

# Age feature omzetten in een nieuwe AgeBand feature, waarbij de Age wordt ingedeeld in 5 leeftijdsgroepen:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

# ... en gelijk `Age` omzetten in ordinale waarden:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
# train_df.head()

# AgeBand was een tijdelijke feature welke we nu niet meer nodig hebben:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
# train_df.head()

# ———————————————————————
# Toevoegen van een nieuwe feature `FamilySize`
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

# toevoegen we een feature `IsAlone` om aan te geven 
# of een persoon alléén op de boot zit of niet:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

# De nieuwe feature `IsAlone` gebruiken in plaats van 
# `FamilySize`, `Parch` en `SibSp`, welke we ook uit 
# de set verwijderen:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]
# train_df.head()

# ———————————————————————
# toevoegen `Age*Class`
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

# ———————————————————————
# Embarked's ontbrekende waarden aanvullen:
freq_port = train_df.Embarked.dropna().mode()[0]

for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

# categorische waarden in `Embarked` omzetten naar numeriek:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

# train_df.head()

# ———————————————————————
# Aanvullen van `Fare` met de mediaan:
print("2022-1026 PP TODO: Fare analysis - checken!! ")
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
# 2022-1026 PP: moet vorige regel niet zijn de volgende:
# for dataset in combine:
#     dataset['Fare'].fillna(dataset['Fare'].dropna().median(), inplace=True)
# test_df.head()

# Voor Fare gaan we een bandbreedte definieren:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

# ... en omzetten naar numerieke waarden:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
#train_df.head(10)
# En ook de test set:
#test_df.head(10)

# ———————————————————————
print('\nFeature selection done:')
print(train_df.head())
print('-'*40)
print(test_df.head())

2022-1026 PP TODO: Fare analysis - checken!! 

Feature selection done:
   Survived  Pclass  Sex  Age  Fare  Embarked  Title  IsAlone  Age*Class
0         0       3    0    1     0         0      1        0          3
1         1       1    1    2     3         1      3        0          2
2         1       3    1    1     1         0      2        1          3
3         1       1    1    2     3         0      3        0          2
4         0       3    0    2     1         0      1        1          6
----------------------------------------
   PassengerId  Pclass  Sex  Age  Fare  Embarked  Title  IsAlone  Age*Class
0          892       3    0    2     0         2      1        1          6
1          893       3    1    2     0         0      3        0          6
2          894       2    0    3     1         2      1        1          6
3          895       3    0    1     1         0      1        1          3
4          896       3    1    1     1         0      3        0      

C:\Users\hq0132410\AppData\Local\Temp\ipykernel_12948\1191931430.py:76: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)
C:\Users\hq0132410\AppData\Local\Temp\ipykernel_12948\1191931430.py:140: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the orig

## We zijn er ...

Het heeft even geduurd maar eindelijk zijn de datasets klaar om te gebruiken voor een voorspelling.

Check punten dataset (train en test) dat:
- testdata is exclusief `survived`
- alle waarden (train en test) numeriek zijn.

---

In [6]:
# splits features and targets
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 8), (891,), (418, 8))

We gebruiken nog eerst een regressie voor het valideren van onze aannames voor het aanmaken van nieuwe features en om een overzicht te krijgen welke features sterk bijdragen aan het wel of niet overleven.

In [7]:
logreg = LogisticRegression(solver="lbfgs")
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
1,Sex,2.201445
5,Title,0.397484
2,Age,0.286911
4,Embarked,0.261583
6,IsAlone,0.126942
3,Fare,-0.086368
7,Age*Class,-0.310963
0,Pclass,-0.750392


Hierboven is te zien dat als de waarde van `Sex` verhoogt (0/man naar 1/vrouw), de survival rate het sterkste toeneemt (kans op overleven). Andersom, als de waarde voor `Pclass` hoger wordt, dan wordt neemt de overlevingskans het sterkste af. De zelf toegevoegde feature `Age*Class` is ook een goede (negatieve) voorspeller voor de overlevingskans

### Support Vector Machines

Nu de voorspelling met lineaire SVM

In [8]:
svc = SVC(kernel='linear')
#svc = SVC(kernel="rbf", gamma="auto")
# 'rbf' geeft iets grotere score (83%)

svc.fit(X_train, Y_train)  # train

Y_train_pred=svc.predict(X_train)  # voorspellen
Y_pred = svc.predict(X_test)  # testen

acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
print (acc_svc)

78.68


### Mogelijk volgende stappen

- Verbeteren van bovenstaande oplossing.
- Denk dan aan het toepassen van cross validation.
- Denk dan aan parameter tuning.
- Andere (SVG)kernels?
- Zou jij de ontbrekende data op een andere manier aanvullen?

---